# 1. 전국 범위의 데이터들을 부산관련 데이터들로 추출
 - #### <strong>1.1 부산 과거 침수 이력 추출</strong>
 - #### <strong>1.2 부산 과거 침수 이력에 대한 시간별 강수량 데이터 추출</strong>
 - #### <strong>1.3 util용 부산 법정동 코드 정리</strong>
 - #### <strong>1.4 부산 과거 침수 위치 Code 및 지역이름 추가</strong>

---
## 1.1 부산 과거 침수 이력 추출
 - FL_DATAUPMNG.csv 파일은 전국 침수 데이터
 - FL_DATAUPMNG column 의 ADM_CD가 법정동 코드
 - 부산 법정동 코드는 '26'으로 시작

In [1]:
import pandas as pd

In [2]:
busan_fl_data = pd.read_csv('../data/rawdata/FL_DATAUPMNG.csv')

- ADM_CD type을 int형에서 str형으로 바꾼 후 26으로 시작하는 데이터를 추출

In [3]:
busan_fl_data['ADM_CD'] = busan_fl_data['ADM_CD'].apply(str)
busan_fl_data = busan_fl_data[busan_fl_data['ADM_CD'].str.startswith('26')].reset_index(drop=True)

<br></br>

---
## 1.2 부산 과거 침수 이력에 대한 시간별 강수량 데이터 추출
 - FL_TIMERAIN.csv는 과거 침수 이력의 기간 동안의 시간별 강수량 데이터를 가지고 있다.
 - FL_DATAUPMNG.csv 파일과 SEQ(사건 순서)로 묶여 있다.
 - 부산 과거 침수 이력 data의 SEQ(사건 순서)에 따른 강수량 데이터를 추출한다.
 1. 부산 과거 침수 이력 data로 부터 사건 순서에 대한 SEQ 정보를 추출한다.
 2. 이후 FL_TIMERAIN.csv에서 해당하는 사건 SEQ에 대한 정보만 추출한다.

In [4]:
seq = busan_fl_data['SEQ'].unique()

In [5]:
rain_data = pd.read_csv('../data/rawdata/FL_TIMERAIN.csv')

In [6]:
busan_rain_data = rain_data[rain_data['SEQ'].isin(seq)].reset_index(drop=True)

In [7]:
busan_rain_data.head()

,ADM_CD,HOUR_RAINFALL,SEQ,PRECI_YYMMDDHH,PRECI_HH
0,2629000000,0.0,30907,20110727,1
1,2629000000,0.0,30907,20110727,2
2,2629000000,3.0,30907,20110727,3
3,2629000000,11.5,30907,20110727,4
4,2629000000,0.0,30907,20110727,5


In [8]:
busan_rain_data.to_csv('../data/processing_data/1st/busan_rain_data.csv', index=False, encoding='utf-8')

---
## 1.3 util용 부산 코드 정리
  - 사용할 코드는 시군구 또는 읍면동 코드이므로 n2, n3 값만 존재하는 컬럼만 사용한다.

In [9]:
busan_code = pd.read_excel('../data/rawdata/busan_code_data.xls')

In [10]:
busan_code[['n1','n2','n3','n4']] = pd.DataFrame(busan_code['법정동명'].str.split(' ',3).tolist())

In [11]:
a = busan_code['n2'].notnull()

In [12]:
b = busan_code['n4'].isnull()

In [13]:
busan_code = busan_code[a & b]
busan_code = busan_code.drop(columns=['n1','n4','법정동명']).reset_index(drop=True)

In [14]:
busan_code = busan_code.rename(columns={'법정동코드':'CODE','n2':'ZONE','n3':'DONG'})

In [15]:
busan_code

,CODE,ZONE,DONG
0,2611000000,중구,None
1,2611010100,중구,영주동
2,2611010200,중구,대창동1가
3,2611010300,중구,대창동2가
4,2611010400,중구,중앙동1가
...,...,...,...
203,2671025000,기장군,기장읍
204,2671025300,기장군,장안읍
205,2671025600,기장군,정관읍
206,2671031000,기장군,일광면


In [16]:
busan_code['CODE'] = busan_code['CODE'].apply(str)

In [17]:
busan_code.to_csv('../data/processing_data/2nd/busan_code.csv',  index=False, encoding='utf-8')

---
## 1.4 부산 과거 침수 위치 Code 및 지역이름 추가
 - 정리한 부산 과거 침수 이력과 부산 법정동 코드를 결합하여 저장한다.

In [18]:
busan_flood = pd.merge(busan_fl_data, busan_code, left_on='ADM_CD', right_on='CODE', how='left')

In [19]:
busan_flood.to_csv('../data/processing_data/1st/busan_flood_base.csv',encoding='utf-8',index=False)

정리 
---
 만들어진 데이터
 - 부산 지역 강수량 busan_rain_base.csv
 - 부산 과거침수이력 데이터 + 불완전한 행정동 코드 busan_flood_base.csv
 - 부산 법정동 코드(시군구 및 읍면동) busan_code.csv